# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#Create a Keyspace 
session.execute("""Create keyspace if not exists sparkify
                with replication =
                {'class': 'SimpleStrategy', 'replication_factor': 1}""");

#### Set Keyspace

In [7]:
#Set KEYSPACE to the newly created keyspace above

session.set_keyspace("sparkify")

### Now we need to create tables to run the following queries. With Apache Cassandra we model the database tables on the queries we want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## First we create the required CQL statements for creating/dropping tables and inserting/selecting the data required for all the questions.

### Create Queries for Q1:  
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [8]:
## The table structure is based on the data required, 
## session_id and item_in_session are selected as the partition key and clustering column 
## as they provide a unique combination for the table and are the ones used in the where statment

create_query1 = """create table if not exists song_length_table
                                                (
                                                 session_id int,
                                                 item_in_session int,
                                                 artist_name text, 
                                                 song_name text,
                                                 song_length float,
                                                 primary key (session_id, item_in_session)
                                                )"""

drop_query1 = "drop table if exists song_length_table"

insert_query1 = """insert into song_length_table 
                            (
                             session_id,
                             item_in_session,
                             artist_name,
                             song_name,
                             song_length
                            ) 
                    values (%s, %s, %s, %s, %s)"""

select_query1 = """select 
                        artist_name,
                        song_name,
                        song_length
                   from 
                        song_length_table 
                   where 
                        session_id = 338 
                    and item_in_session = 4"""

### Create Queries for Q2: 
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [9]:
## The table structure is based on the data required, 
## user_id is the primary key and session_id and item_in_session are selected as clustering columns 
## as they provide a unique combination for the table and are the ones used in the where statment 
## item_in_session is used to allow for sorting

create_query2 = """create table if not exists sorted_song_table
                                                (
                                                 user_id int,
                                                 session_id int,
                                                 item_in_session int, 
                                                 artist_name text,
                                                 song_name text,
                                                 first_name text,
                                                 last_name text,
                                                 primary key (user_id, session_id, item_in_session)
                                                )"""

drop_query2 = "drop table if exists sorted_song_table"

insert_query2 = """insert into sorted_song_table 
                            (
                             user_id,
                             session_id,
                             item_in_session,
                             artist_name,
                             song_name,
                             first_name,
                             last_name
                            ) 
                    values (%s, %s, %s, %s, %s, %s, %s)"""

select_query2 = """select 
                        artist_name,
                        song_name,
                        first_name,
                        last_name
                   from 
                        sorted_song_table 
                   where 
                        user_id = 10 
                    and session_id = 182"""

### Create Queries for Q3:
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [10]:
## The table structure is based on the data required, 
## song name is the partition key so it can be used in the where statement 
## and user_id is selected as clustering to provide a unique identifier for user 
## (instead of the names in case the first and last name combination is repeated)
## and since we only care if the user played the song or not, this is a unique enough key for our table

create_query3 = """create table if not exists user_table 
                                                    (
                                                     song_name text,
                                                     user_id int, 
                                                     first_name text,
                                                     last_name text,
                                                     primary key (song_name, user_id)
                                                    )"""

drop_query3 = "drop table if exists user_table"

insert_query3 = """insert into user_table
                            (
                             song_name, 
                             user_id,
                             first_name,
                             last_name
                            )
                    values (%s, %s, %s, %s)"""

select_query3 = """select 
                        first_name,
                        last_name
                   from 
                        user_table
                   where song_name = 'All Hands Against His Own'"""

In [11]:
## Creaete tables using the previously written CQL statements 

session.execute(create_query1);
session.execute(create_query2);
session.execute(create_query3);

In [12]:
### This part reads the combined csv file 
file = 'event_datafile_new.csv'

## Here we read the file line by line and use its contents for all the insert CQL statements we have to decrease code repetition
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Insert query for Q1:
        session.execute(
                        insert_query1, 
                        (
                         int(line[8]),
                         int(line[3]), 
                         line[0],
                         line[9],
                         float(line[5])
                        )
                       )
        #Insert query for Q2:
        session.execute(
                        insert_query2,
                        (
                         int(line[10]), 
                         int(line[8]),
                         int(line[3]),
                         line[0],
                         line[9], 
                         line[1],
                         line[4]
                        )
                       )
        #Insert query for Q3:
        session.execute(
                        insert_query3,
                        (
                         line[9],
                         int(line[10]), 
                         line[1],
                         line[4]
                        )
                       )

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
## SELECT statement to verify the data was entered into the table
result = session.execute(select_query1)

## show result as dataframe 
pd.DataFrame([item for item in result], columns = ['Artist Name', 'Song Name', 'Song Length'])

,Artist Name,Song Name,Song Length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


In [14]:
##SELECT statement to verify the data was entered into the table
result = session.execute(select_query2)

## show result as dataframe 
pd.DataFrame([item for item in result], columns = ['Artist Name', 'Song Name', 'First Name', 'Last Name'])

,Artist Name,Song Name,First Name,Last Name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [15]:
## Add in the SELECT statement to verify the data was entered into the table
result = session.execute(select_query3)

## show result as dataframe 
pd.DataFrame([item for item in result], columns=['First Name', 'Last Name'])

,First Name,Last Name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [16]:
## Drop the tables before closing out the sessions

In [17]:
session.execute(drop_query1);
session.execute(drop_query2);
session.execute(drop_query3);

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()